In [1]:
#######################################################################
##  ECS 251 Spring 2016 Final Project Code
##  Guicheng Wu
##  Xingtai Li

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from urllib import urlopen
import logging
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import pylab
import copy
import random
import difflib
from sklearn import svm, naive_bayes,linear_model, neighbors, preprocessing, cross_validation,decomposition,tree
from sklearn.metrics.pairwise import rbf_kernel,sigmoid_kernel,laplacian_kernel,safe_sparse_dot,check_pairwise_arrays
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.utils.extmath import row_norms, safe_sparse_dot
import math
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,hamming_loss

############################################################################
# Create a search engine object
class se:
    def __init__(self,adjacent_list,node_list,key):
        self.G = self.read_links(adjacent_list)
        self.N = self.read_nodes(node_list)
        self.G.remove_node(-1)
        #self.delete_empty()
        self.keyword = key
        self.directory_text = 'basketball/'
        # write the webpage content to text files
        #directory = 'basketball/'
        #write_webpage_txt(directory,self.__N)
        self.tfidf,self.countvect,self.X = self.TFIDF_weight()
        self.tfidf_key = self.TFIDF_keyword()
        self.pr = self.page_rank(0.9).values()
        self.hub_weights = self.hits()[0].values()
        self.authorities_weights = self.hits()[1].values()
        self.title_count = self.title_key()
        self.construct_feature_matrix()
        
    # Construct the link graph
    def read_links(self,name):
        # read adjacent list file
        with open(name) as f:
            lists = []
            for line in f:
                lists.append(line.replace(":","").rstrip())
        # Form graph
        G = nx.parse_adjlist(lists,create_using=nx.DiGraph(),nodetype=int)
        return G

    # Read the node file and construct a dictionary of nodes
    def read_nodes(self,name):
        with open(name) as f:
            number_of_node = int(next(f).rstrip())
            next(f)
            D = {}
            for i in range(number_of_node):
                d = {}
                idd = int(next(f).split(' ')[0])
                d['url'] = next(f).rstrip()
                d['title'] = next(f).rstrip()
                links = next(f).rstrip().split(' ')
                d['in'] = int(links[0])
                d['out'] = int(links[1])
                D[idd] = d
                if i != number_of_node - 1:
                    next(f)
        return D
    
    # Get the page rank scores
    def page_rank(self,alpha):
        return nx.pagerank(self.G, alpha=alpha)

    # Get the hits scores
    def hits(self):
        return nx.hits(self.G)

    # Number of terms in the title of page
    def title_key(self):
        
        title_keys = []
        for key in self.N:
            title_words = self.N[key]['title'].split()
            http = self.N[key]['url'].split()
            matching = [s for s in title_words if self.keyword in s]
            matching_2 = [s for s in http if self.keyword in s]
            title_keys.append(len(matching)+len(matching_2))
#         title_keys = np.matrix(title_keys,dtype='float')
#         return preprocessing.MinMaxScaler().fit_transform(title_keys)
        return title_keys
        
    # Read the webpage links
    def retrive_html_content(url):
        try:
            html = urlopen(url).read()
        except:
            logging.error('Data cannot retrieved in the \nURL: %s',url)
            html = ""  

        soup = BeautifulSoup(html)

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return text

    # Write the web content to the specified directory
    def write_webpage_txt(self,directory):
        for i in range(3527,len(N)):
            link = N[i]['url']
            page_content = retrive_html_content(link).encode("utf8")
            web_file =directory +  str(i) + '.txt'
            file_stream = open(web_file, 'w')
            file_stream.write(page_content)

    #build TDIDF matrix
    def TFIDF_weight(self):
        corpus = []
        webpage_count = len(self.N)
        for i in range(webpage_count):
            web_file = self.directory_text + str(i) + '.txt'
            file_stream = open(web_file, 'r')
            content = file_stream.read()
            corpus.append(content)

        vectorizer = CountVectorizer(min_df=1)
        countvect = vectorizer.fit(corpus)
        X = vectorizer.fit_transform(corpus)

        transformer = TfidfTransformer()
        #build TFIDF matrix
        tfidf = transformer.fit_transform(X)
        return tfidf,countvect,X

    # Calculate the weight of tfidf 
    def TFIDF_keyword(self):
        #get all features
        words = self.countvect.get_feature_names()
        #if keywords exist in features, get the TFIDF weight; else set TFIDF weight as 0
        tf_new = np.zeros((len(self.G),1))
        list1 = []
        for i in range(self.tfidf.shape[1]):
            if self.keyword in words[i]:
                list1.append(i)
                tf_new+=self.tfidf[:,i]
        
        return tf_new
    
    # Construct the feature matrix 
    def construct_feature_matrix(self):
        feature_matrix = np.column_stack((self.pr,self.hub_weights,self.authorities_weights,self.title_count,self.tfidf_key))
        feature_file = self.keyword+'.txt'
        np.savetxt(feature_file,feature_matrix)
        
    # Retrieve the top words from the tfidf
    def retrieve_top_words(self,size):
        words = self.c_1.get_feature_names()
        word_count = ((self.X.sum(axis=0).T).tolist())
        indicies = [i[0] for i in sorted(enumerate(word_count), key=lambda x:x[1])]
        top_indicies = indicies[len(word_count)-size:len(word_count)]
        top_words = []
        for i in top_indicies:
            top_words.append(words[i])
        return top_words
    
    # Get the prediction score from the tfidf_key
    def get_prediction(self,key_list):
        names = self.countvect.get_feature_names()
        y_pred = np.zeros((len(self.N),len(key_list)))
        for i in range(self.X.shape[1]):
            for j in range(len(key_list)):
                if key_list[j] in names[i]:
                    for k in range(len(self.N)):
                        if y_pred[k][j] == 0:
                            y_pred[k][j] = 1
        return y_pred
    
    # Label the response
    def label_title(self):
        label_y = []
        i = 0
        for key in self.N:
            title_words = self.N[key]['title'].split()
            if self.keyword in title_words:
                label_y.append(1)
            else:
                label_y.append(0)
            i += 1 
        label_y = np.matrix(label_y).T
        return label_y
    
###################################################
## Label y
def label_y(engine,num):
    count = []
    names = engine.countvect.get_feature_names()
    for i in range(engine.X.shape[1]):
        if engine.keyword in names[i]:
            count.append(i)
    V = engine.X
    y_news = np.zeros((len(engine.G),1))
    for i in range(len(count)):
        y_news += V[:,count[i]]

    for i in range(len(y_news)):
        if y_news[i] > num:
            y_news[i] = 1
        else:
            y_news[i] = 0
    y_label = engine.label_title()
    y_news += y_label
    cc = 0
    for i in range(len(y_news)):
        if y_news[i] != 0:
            cc+=1
            y_news[i] = 1
    np.savetxt('ncaa_y1.txt',y_news,fmt='%i')
    return cc

##########################################################################
## Create a ranking object which return a list of hyperlink according to their revelance to query
class ranking():
    def __init__(self,q,engine):
        self.q = q
        self.engine = engine
        self.feature = self.get_feature()
        self.label = self.get_label()
        self.G, self.N, self.index,self.feature_t = self.create_graph()
        self.score = self.ranking_score()
        
    # Get feature matrix
    def get_feature(self):
        with open('dataset/'+self.q+'.txt') as f:
            feature = []
            for line in f:
                feature.append(line.rstrip().split(' '))
            feature = np.array(feature,dtype='float')
        return feature
    # Get label of pages
    def get_label(self):
        with open('dataset/'+self.q+'_y2.txt') as f:
            label = []
            for line in f:
                label.append(int(line.rstrip()))
        return label
    # Create Gragh
    def create_graph(self):
        index_d = []
        index_k = []
        N = self.engine.N
        G = self.engine.G
        feature_t = []
        for i in range(len(self.label)):
            if self.label[i] == 0:
                index_d.append(i)
                del N[i]
            else:
                index_k.append(i)
                feature_t.append(np.append((i),self.feature[i]))
        G.remove_nodes_from(index_d)
        return G,N,index_k,np.array(feature_t)
    # Print the ranking of hyperlinks
    def ranking_score(self):
        result = []

        rank = np.zeros((len(self.N),7))
        rank[:,0] = self.feature_t[:,0]
        
        num_pr = np.argsort(self.feature_t[:,1])
        num_aut = np.argsort(self.feature_t[:,2])
        num_hub = np.argsort(self.feature_t[:,3])
        num_title = np.argsort(self.feature_t[:,4])
        num_content = np.argsort(self.feature_t[:,5])
        
        for j in range(len(num_title)):
            rank[num_pr[j]][1] = j+1
            rank[num_aut[j]][2] = j+1
            rank[num_hub[j]][3] = j+1
            rank[num_title[j]][4] = j+1
            rank[num_content[j]][5] = j+1
        
        tnum = rank.shape[0]
        for k in range(rank.shape[0]):
            rank[k][6] = np.multiply(10,[(tnum-rank[k][1])/tnum])+np.multiply(10,[(tnum-rank[k][2])/tnum])
            +np.multiply(10,[(tnum-rank[k][3])/tnum])+np.multiply(100,[(tnum-rank[k][4])/tnum])
            +np.multiply(10,[(tnum-rank[k][5])/tnum])
        indexes = []
        for key in self.N:
            if self.q in self.N[key]['url']:
                indexes.append(key)
        for u in range(len(indexes)):
            for o in range(rank.shape[0]):
                if rank[o][0] == indexes[u]:
                    rank[o][6] = np.multiply(10,rank[o][6])        
        final = np.argsort(rank[:,6])
        for i in range(tnum):
            result.append(self.N[self.feature_t[final[i],0]]['url'])
        return result
    # Save the txt file    
    def print_score(self):
        np.savetxt('result/'+self.q+ '_score.txt',self.score,fmt="%s")

###################################################################################
def generate_ranking_scores():
    for i in range(len(query)):
        engine = se('All/basketball/graph/adj_list','All/basketball/graph/nodes',query[i])
        r = ranking(query[i],engine)
        #r.print_score()
        #save_graph(r.G,'result/'+query[i]+'.png')
        
###################################################################################
# draw the graph
def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig
#save_graph(r.G,'my_graph_1.pdf')

###################################################################################
## Calculate page relevance 
class page_relevance():
    
    def __init__(self,engine):
        self.engine = engine
        self.page_1, self.page_2 = self.get_pages()
        self.score = self.check()
        #self.print_scores()
        #self.print_example()
    # Randomly generate two pages
    def get_pages(self):
        page = random.sample(self.engine.N,2)
        return page[0],page[1]
    # Check the decision tree, return a score of two pages
    def check(self):
        if nx.has_path(self.engine.G,self.page_1,self.page_2):
            length,path = nx.bidirectional_dijkstra(self.engine.G,self.page_1,self.page_2)
        else:
            length = 0
        value = 0
        score_2 = self.text_match()
        if length > 0 & length < 10:
            value = (11.0-float(length))/10.0
        elif length >= 10:
            score_l = 1.0/float(length)
            value = score_1 + score_2
        else:
            value = score_2
        return value
    # Texutal Relevance       
    def text_match(self):
        score = 0.0
        web_1 = self.engine.N[self.page_1]['url'][7:-5]
        web_2 = self.engine.N[self.page_2]['url'][7:-5]
        title_1 = self.engine.N[self.page_1]['title']
        title_2 = self.engine.N[self.page_2]['title']
        file_name_1 = 'basketball/' + str(self.page_1) + '.txt'
        file_stream = open(file_name_1, 'r')
        content_1 = file_stream.read()
        file_name_2 = 'basketball/' + str(self.page_2) + '.txt'
        file_stream = open(file_name_2, 'r')
        content_2 = file_stream.read()
        
        web_score = difflib.SequenceMatcher(None, web_1,web_2).ratio()
        title_score = difflib.SequenceMatcher(None, title_1,title_2).ratio()
        
        if (web_score > 0.5) | (title_score > 0.5):
            score = max(web_score,title_score)
        else:
            if (len(content_1) < 100) | (len(content_2)<100):
                score = min(web_score,title_score)
            else:
                corpus = []
                corpus.append(content_1)
                corpus.append(content_2)
                vectorizer = CountVectorizer(min_df=1)
                X = vectorizer.fit_transform(corpus)
                X = np.squeeze(np.asarray(X.todense()))
                sort_1 = np.argsort(X[0])
                sort_2 = np.argsort(X[1])
                count_s = 0
                same = []
                for i in range(len(sort_1)):
                    for j in range(len(sort_2)):
                        if sort_1[i] ==  sort_2[j]:
                            same.append(sort_1[i])
                for k in range(len(same)):
                    if (X[0][sort_1[same[k]]] > 1) & (X[1][sort_2[same[k]]] > 1):
                        count_s += 1
                count_score = float(count_s) / 10.0
                score = min(web_score,title_score,count_s) 
        return score
    # Good mathch example
    def good_match(self):
        web_1 = 'http://www.usatoday.com/sports/basketba/skm/acc/skma08.htm'
        web_2 = 'http://www.usatoday.com/sports/basketba/skm/pac10/skmg09.htm'
        web_3 = 'http://gostanford.fansonly.com/sports/w-baskbl/stats/teamstat.html'
        web_4 = 'http://gostanford.fansonly.com/sports/w-baskbl/archive/stan-w-baskbl-archive.html'  
        web_score_1 = difflib.SequenceMatcher(None, web_1,web_2).ratio()
        web_score_2 = difflib.SequenceMatcher(None, web_3,web_4).ratio()
        return web_score_1,web_1,web_2,web_score_2,web_3,web_4
    # Good link example
    def good_link(self):
        length,path = nx.bidirectional_dijkstra(self.engine.G,0,195)
        web_1 = self.engine.N[0]['url']
        web_2 = self.engine.N[195]['url']
        score = (11.0-float(length))/10.0 - (0.1 * difflib.SequenceMatcher(None, web_1,web_2).ratio())
        return score,web_1,web_2
    # Print the scores
    def print_scores(self):
        content = []
        content.append('Web page 1: ' + self.engine.N[self.page_1]['url'])
        content.append('Web page 2: ' + self.engine.N[self.page_2]['url'])
        content.append('Page Relevance Score: ' + str(self.score))
        return content
    # Print example
    def print_example(self):
        content = []
        good_m = self.good_match()
        good_d = self.good_link()
        content.append('Web page 1: ' + good_m[1])
        content.append('Web page 2: ' + good_m[2])
        content.append('Page Relevance Score: ' + str(good_m[0]))
        content.append('')
        content.append('Web page 1: ' + good_m[4])
        content.append('Web page 2: ' + good_m[5])
        content.append('Page Relevance Score: ' + str(good_m[3]))
        content.append('')
        content.append('Web page 1: ' + good_d[1])
        content.append('Web page 2: ' + good_d[2])
        content.append('Page Relevance Score: ' + str(good_d[0]))
        return content

##########################################################
## Classification of page 
class classifier:
    def decision_rule(self, hat_Y):
        for i in np.arange(hat_Y.shape[0]):
            if hat_Y[i] > 0.5:
                hat_Y[i] = 1
            else:
                hat_Y[i] = 0
        return hat_Y

    def ridge_regression(self, alpha=10):
        ridge = linear_model.Ridge(alpha)
        ridge.fit(self.X_train, self.Y_train)
        return ridge

    def kNearestNeighbors(self, k=5):
        knn = neighbors.KNeighborsRegressor(k)
        knn.fit(self.X_train, self.Y_train)
        return knn

    #SVM kernel_type: linear, polynomial, rbf, sigmoid, p
    def linear_svm(self):
        linearsvm = svm.SVC(kernel='linear')
        linearsvm.fit(self.X_train, self.Y_train)
        return linearsvm
    
    def bayes_gaussian(self):
        gaussianbayes = naive_bayes.GaussianNB()
        gaussianbayes.fit(self.X_train, self.Y_train)
        return gaussianbayes

    def logistic_regression(self):
        logistic = linear_model.LogisticRegression()
        logistic.fit(self.X_train, self.Y_train)
        return logistic

    def linearDiscrminat(self):
        linearDA = LDA()
        linearDA.fit(self.X_train, self.Y_train)
        return linearDA

    def lasso_regression(self):
        lasso_regr = linear_model.Lasso(alpha=1)
        lasso_regr.fit(self.X_train, self.Y_train)
        return lasso_regr

    def decision_tree(self):
        decisionTree = tree.DecisionTreeClassifier()
        decisionTree.fit(self.X_train, self.Y_train)
        return decisionTree

    def predict_ridge_regression(self):
        hat_Y = self.ridge_regression().predict(self.X_test)
        return self.decision_rule(hat_Y)

    def predict_knn(self, k=5):
        hat_Y = self.kNearestNeighbors().predict(self.X_test)
        return self.decision_rule(hat_Y)

    def predict_linearsvm(self):
        hat_Y = self.linear_svm().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)
    
    def predict_bayesgaussin(self):
        hat_Y = self.bayes_gaussian().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)  

    def predict_logistic(self):
        hat_Y = self.logistic_regression().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)

    def predict_LDA(self):
        hat_Y = self.linearDiscrminat().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)

    def predict_lasso(self):
        hat_Y = self.lasso_regression().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)

    def predict_decisionTree(self):
        hat_Y = self.decision_tree().predict(self.X_test)
        hat_Y = hat_Y.reshape(hat_Y.shape[0], 1)
        return self.decision_rule(hat_Y)

    def correct_rate(self, hat_Y):
        return accuracy_score(self.Y_test, hat_Y)
    
    def test_error(self, hat_Y):
        return hamming_loss(self.Y_test, hat_Y)
    
    def result_report(self, hat_Y):
        target_names = ['Irrevelence', 'Relevance']
        return classification_report(self.Y_test, hat_Y, target_names=target_names)
    
    def pca_graph(self, dimension):
        #X = laplacian_kernel(self.x_matrix)
        #X = polynomial_kernel(self.x_matrix)
        X = self.x_matrix
        #X = polynomial_kernel(X)
        if dimension == 2:
            pca = decomposition.PCA(n_components=2)
            pca.fit(X)
            X = np.array(pca.transform(X))
            data_fig1 = plt.figure(1, figsize=(8, 6))
            plt.clf()
            #Plot the training points
            plt.scatter(X[:, 0], X[:, 1], c=np.asarray(self.label_y), cmap=plt.cm.Paired)
            plt.xlabel('Projection Vector 1')
            plt.ylabel('Projection Vector 2')
            plt.show()
            data_fig1.savefig('data_2d.png')
        elif dimension == 3:
            pca = decomposition.PCA(n_components=3)
            pca.fit(self.x_matrix)
            X = np.array(pca.transform(self.x_matrix))
            
            data_fig2 = plt.figure(2)
            ax2 = data_fig2.add_subplot(111, projection='3d')
            ax2.scatter(np.asarray(X[:,0]), np.asarray(X[:,1]), 
                        np.asarray(X[:, 2]), c=np.asarray(self.label_y), cmap=plt.cm.Paired)
            plt.show()
            data_fig2.savefig('data_3d.png')
            
    def __init__(self):
        feature_file = 'dataset/women.txt'
        label_file = 'dataset/women_y1.txt'
        x_matrix = np.loadtxt(feature_file, dtype='float')
        label_y = np.loadtxt(label_file, dtype='float')
        self.x_matrix = np.matrix(x_matrix)
        self.label_y = np.matrix(label_y).T
        
        #data_matrix = np.concatenate((x_matrix, label_y), axis=1)
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.x_matrix, 
                                                                                self.label_y, test_size=.4)

#############################################################
def classification_result():
    instance = classifier()
    ridge_reg_hatY = instance.predict_ridge_regression()
    knn_hatY = instance.predict_knn()
    svm_hatY = instance.predict_linearsvm()
    bayes_hatY = instance.predict_bayesgaussin()
    logistic_hatY = instance.predict_logistic()
    lda_hatY = instance.predict_LDA()
    lasso_hatY = instance.predict_lasso()
    desTree_hatY = instance.predict_decisionTree()

    print 'ridge regression'
    print instance.result_report(ridge_reg_hatY)

    print 'knn'
    print instance.result_report(knn_hatY)

    print 'linear support vector machine'
    print instance.result_report(svm_hatY)

    print 'bayes gaussian'
    print instance.result_report(bayes_hatY)

    print 'logistic regression'
    print instance.result_report(logistic_hatY)

    print 'LDA'
    print instance.result_report(lda_hatY)

    print 'lasso'
    print instance.result_report(lasso_hatY)

    print 'decision tree'
    print instance.result_report(desTree_hatY)
    
###################################################################################
query = ['news','season','sports','team','NBA','women','schedule','college','NCAA','player']

engine = se('All/basketball/graph/adj_list','All/basketball/graph/nodes','basketball')
relevance_pbject = page_relevance(engine)
test = relevance_pbject.print_scores()
classification_result()
test

/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


ridge regression
             precision    recall  f1-score   support

Irrevelence       0.85      1.00      0.92      1998
  Relevance       1.00      0.19      0.32       422

avg / total       0.88      0.86      0.82      2420

knn
             precision    recall  f1-score   support

Irrevelence       0.99      1.00      0.99      1998
  Relevance       0.99      0.94      0.96       422

avg / total       0.99      0.99      0.99      2420

linear support vector machine
             precision    recall  f1-score   support

Irrevelence       0.89      1.00      0.94      1998
  Relevance       1.00      0.41      0.58       422

avg / total       0.91      0.90      0.88      2420

bayes gaussian
             precision    recall  f1-score   support

Irrevelence       0.99      1.00      0.99      1998
  Relevance       0.98      0.94      0.96       422

avg / total       0.99      0.99      0.99      2420

logistic regression
             precision    recall  f1-score   support



/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/guichengwu/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

['Web page 1: http://www.warwickdc-leisure.co.uk/sports_dev.htm',
 'Web page 2: http://www.miami.com/mld/miamiherald/sports/high_school/4606722.htm',
 'Page Relevance Score: 0.0869565217391']